**Question Number 3: A variation of the problem mentioned in Question 2**

In [ ]:
#IMPORTS
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson
import sys

Continuous Tasks : These are the tasks that have no ends i.e. they don’t have any terminal state.

Discount Factor (ɤ): It determines how much importance is to be given to the immediate reward and future rewards. This basically helps us to avoid infinity as a reward in continuous tasks. It has a value between 0 and 1. A value of 0 means that more importance is given to the immediate reward and a value of 1 means that more importance is given to future rewards. In practice, a discount factor of 0 will never learn as it only considers immediate reward and a discount factor of 1 will go on for future rewards which may lead to infinity. Therefore, the optimal value for the discount factor lies between 0.2 to 0.8.

In [ ]:
#Problem Parameters
max_bikes = 20
discount_rate = 0.9
reward_on_credit = 10
reward_on_moving = -2
reward_on_using_second_parking_lot = -4

Poisson Distribution:
A Poisson distribution is a discrete probability distribution. It gives the probability of an event happening a certain number of times (k) within a given interval of time or space. lamda is the mean





In [ ]:
class poisson_:
    def __init__(self, lmda):
        self.lmda = lmda
        ε = 0.01
        # [α , β] is the range of n's for which the pmf value is above ε
        self.alpha = 0
        state = 1
        self.vals = {}
        summer = 0
        
        while(1):
            if state == 1:
                temp = poisson.pmf(self.alpha, self.lmda) 
                if(temp <= ε):
                    self.alpha+=1
                else:
                    self.vals[self.alpha] = temp
                    summer += temp
                    self.beta = self.alpha+1
                    state = 2
            elif state == 2:
                temp = poisson.pmf(self.beta, self.lmda)
                if(temp > ε):
                    self.vals[self.beta] = temp
                    summer += temp
                    self.beta+=1
                else:
                    break  
            # normalizing the pmf, values of n outside of [α, β] have pmf = 0
        
        added_val = (1-summer)/(self.beta-self.alpha)
        for key in self.vals:
            self.vals[key] += added_val
        
            
    def f(self, n):
        try:
            Ret_value = self.vals[n]
        except(KeyError):
            Ret_value = 0
        finally:
            return Ret_value  
        

In [ ]:
#A class holding the properties of a location together
class location:
    def __init__(self, req, ret):
        self.alpha = req                            #value of lambda for requests
        self.beta = ret                             #value of lambda for returns
        self.poissonalpha = poisson_(self.alpha)
        self.poissonbeta = poisson_(self.beta)

In [ ]:
#Location initialisation
A = location(3,3)
B = location(4,2)

In [ ]:
#Initializing the value and policy matrices. Initial policy has zero value for all states.
#value is a dp matrix of size 21x21
value = np.zeros((max_bikes+1, max_bikes+1))
policy = value.copy().astype(int)

In [ ]:
#action is basically taking bikes from one location to the other
def apply_action(state, action):
    return [max(min(state[0] - action, max_bikes),0) , max(min(state[1] + action, max_bikes),0)]

In [ ]:
def expected_reward(state, action):
    global value
    """
    state  : It's a pair of integers, # of bikes at A and at B
    action : # of bikes transferred from A to B,  -5 <= action <= 5 
    """
    si = 0 #reward
    new_state = apply_action(state, action)    
    
    # adding reward for moving cars from one location to another (which is negative) 
    if action <= 0:
        si = si + max_bikes * abs(action)
    else:
        si = si + max_bikes * (action - 1)  #one car is moved by one of Jack's employees for free
    
    # adding reward for second parking lot (which is also negative)
    if new_state[0] > 10:
        si = si + reward_on_using_second_parking_lot   
    if new_state[1] > 10:
        si = si + reward_on_using_second_parking_lot
    
    # there are four discrete random variables which determine the probability distribution of the reward and next state
    for Aalpha in range(A.poissonalpha.alpha, A.poissonalpha.beta):
        for Balpha in range(B.poissonalpha.alpha, B.poissonalpha.beta):
            for Abeta in range(A.poissonbeta.alpha, A.poissonbeta.beta):
                for Bbeta in range(B.poissonbeta.alpha, B.poissonbeta.beta):
                    """
                    Aα : sample of bikes requested at location A
                    Aβ : sample of bikes returned at location A
                    Bα : sample of s bikes requested at location B
                    Bβ : sample of bikes returned at location B
                    p  : probability of this event happening
                    """
                    
                    # all four variables are independent of each other
                    p = A.poissonalpha.vals[Aalpha] * B.poissonalpha.vals[Balpha] * A.poissonbeta.vals[Abeta] * B.poissonbeta.vals[Bbeta]
                    
                    valid_requests_A = min(new_state[0], Aalpha)
                    valid_requests_B = min(new_state[1], Balpha)
                    
                    rew = (valid_requests_A + valid_requests_B)*(reward_on_credit)
                    
                    #calculating the new state based on the values of the four random variables
                    new_s = [0,0]
                    new_s[0] = max(min(new_state[0] - valid_requests_A + Abeta, max_bikes),0)
                    new_s[1] = max(min(new_state[1] - valid_requests_B + Bbeta, max_bikes),0)
                    
                    #Bellman's equation
                    si += p * (rew + discount_rate * value[new_s[0]][new_s[1]])
                    
    return si

In [ ]:
def policy_evaluation():
    
    global value
    
    # here policy_evaluation has a static variable ε whose values decreases over time
    e = policy_evaluation.e
    policy_evaluation.e /= 10 
    
    while(1):
        delta = 0
        for i in range(value.shape[0]):
            for j in range(value.shape[1]):
                # value[i][j] denotes the value of the state [i,j]
                
                old_val = value[i][j]
                value[i][j] = expected_reward([i,j], policy[i][j])
                
                delta = max(delta, abs(value[i][j] - old_val))
                print('.', end = '')
                sys.stdout.flush()
        print(delta)
        sys.stdout.flush()
    
        if delta < e:
            break

In [ ]:
#initial value of ε
policy_evaluation.e = 50

In [ ]:
def policy_improvement():
    
    global policy
    
    policy_stable = True
    for i in range(value.shape[0]):
        for j in range(value.shape[1]):
            old_action = policy[i][j]
            
            max_act_val = None
            max_act = None
            
            τ12 = min(i,5)       # if I have say 3 cars at the first location, then I can atmost move 3 from 1 to 2
            τ21 = -min(j,5)      # if I have say 2 cars at the second location, then I can atmost move 2 from 2 to 1
            
            for act in range(τ21,τ12+1):
                σ = expected_reward([i,j], act)
                if max_act_val == None:
                    max_act_val = σ
                    max_act = act
                elif max_act_val < σ:
                    max_act_val = σ
                    max_act = act
                
            policy[i][j] = max_act
            
            if old_action!= policy[i][j]:
                policy_stable = False
    
    return policy_stable

In [ ]:
def save_policy():
    save_policy.counter += 1
    ax = sns.heatmap(policy, linewidth=0.5)
    ax.invert_yaxis()
    plt.savefig('policy'+str(save_policy.counter)+'.svg')
    plt.close()
    
def save_value():
    save_value.counter += 1
    ax = sns.heatmap(value, linewidth=0.5)
    ax.invert_yaxis()
    plt.savefig('value'+ str(save_value.counter)+'.svg')
    plt.close()


In [ ]:
save_policy.counter = 0
save_value.counter = 0
while(1):
    policy_evaluation()
    ρ = policy_improvement()
    save_value()
    save_policy()
    if ρ == True:
        break

.........................................................................................................................................................................................................................................................................................................................................................................................................................................................169.90008661446765
.........................................................................................................................................................................................................................................................................................................................................................................................................................................................119.08501115392248
................................................................................